In [1]:
import tensorflow as tf

# Load the data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 64
display_step = 20

# Network Parameters
n_input = 784
n_classes = 10
dropout = 0.8 

In [5]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# define weights and biases
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64])),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wd1': tf.Variable(tf.random_normal([4*4*256, 1024])),
    'wd2': tf.Variable(tf.random_normal([1024, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 10]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([128])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
# define the function

# convolution
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], padding='SAME'), b), name=name)

# maxpooling
def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME', name=name)

# normalization
def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001/9.0, beta=0.75, name=name)

In [14]:
# define Alexnet
def alex_net(_X, _weights, _biases, _dropout):
    # vector transform into matrix
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])
    # convolution layer1
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    # down sampling layer1
    pool1 = max_pool('pool1', conv1, k=2)
    # normalization layer1
    norm1 = norm('norm1', pool1, lsize=4)
    # Dropout layer1
    norm1 = tf.nn.dropout(norm1, _dropout)
    
    # convolution layer2 
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    # down sampling layer2
    pool2 = max_pool('pool2', conv2, k=2)
    # normalization layer2
    norm2 = norm('norm2', pool2, lsize=4)
    # Dropuout layer2
    norm2 = tf.nn.dropout(norm2, _dropout)
    
    # convolution layer3
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    # down samling layer3
    pool3 = max_pool('pool3', conv3, k=2)
    # normalization layer3
    norm3 = norm('norm3', pool3, lsize=4)
    # Dropout layer3
    norm3 = tf.nn.dropout(norm3, _dropout)
    
    # Fully connected
    # transfer features into vector
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]])
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1')
    # fully connected
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2')
    
    # output
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    return out

In [15]:
# build the model
pred = alex_net(x, weights, biases, keep_prob)

# define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [18]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    print( "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))

Iter 1280, Minibatch Loss= 179960.000000, Training Accuracy= 0.23438
Iter 2560, Minibatch Loss= 108522.929688, Training Accuracy= 0.34375
Iter 3840, Minibatch Loss= 79018.421875, Training Accuracy= 0.39062
Iter 5120, Minibatch Loss= 61869.871094, Training Accuracy= 0.42188
Iter 6400, Minibatch Loss= 55646.117188, Training Accuracy= 0.50000
Iter 7680, Minibatch Loss= 38644.687500, Training Accuracy= 0.59375
Iter 8960, Minibatch Loss= 33005.437500, Training Accuracy= 0.62500
Iter 10240, Minibatch Loss= 20184.093750, Training Accuracy= 0.68750
Iter 11520, Minibatch Loss= 16049.242188, Training Accuracy= 0.78125
Iter 12800, Minibatch Loss= 32125.662109, Training Accuracy= 0.54688
Iter 14080, Minibatch Loss= 22253.968750, Training Accuracy= 0.62500
Iter 15360, Minibatch Loss= 18647.613281, Training Accuracy= 0.67188
Iter 16640, Minibatch Loss= 16983.009766, Training Accuracy= 0.76562
Iter 17920, Minibatch Loss= 15953.605469, Training Accuracy= 0.70312
Iter 19200, Minibatch Loss= 14793.30859